# Tutorial: Point Cloud Deep Learing With Forest Lidar Data

By Harry Seely

This is a simple tutorial to get you started with using a point cloud deep learning model to process forest lidar data.

While deep learning is a but tough to use at first, once you get the hang of it you can begin to appreciate the flexibility and power of these models.

If you would like more information on point cloud deep learning, check out my repo [https://github.com/harryseely/DL_Biomass](https://github.com/harryseely/DL_Biomass)


This tutorial was developed using python v3.8

All you need to run it is a python environment setup with jupyter notebook installed.


In [ ]:
#Install requirements
# !pip install torch torchvision torchaudio
# !pip install matplotlib

In [147]:
#Load required modules
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.colors import ListedColormap

## Sample Point Cloud Data

So lets start with some simple point clouds. How about 12 forest plots point clouds each which has 7168 points or 18 pts/m^2  (to keep file size small). It is required for most neural networks that each input file has the same dimensions, which is why the point clouds need to be pre-processed to have the same number of points.

In [ ]:
#Get our files
in_dir = r"data"
glob = "*.npy"

#Read in all files
files = list(Path(in_dir).glob(glob))
files

We have three datasets: train, validation, and test

Lets visualize one of these point clouds.

Each point cloud has 8 columns and 7168 rows. The columns include xyz and other lidar attributes such as intensity and scan angle. 


In [ ]:
#Load point cloud from numpy file
points = np.load("data/NBGOV10160_7168_test_duplicate_or_fps.npy")

# Extract xyz only
xyz = points[:, 0:3]

# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=[30, 30])

# set up the axes for the first plot
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2], c=xyz[:, 2], linewidth=1, s=50)

Ok so its a bit hard to see in this figure but this is a 11.28m (399.7m2) forest plot

For a simple initial modelling task, we can get the model to predict the average height for each plot. Lets calculate that for every plot in our dataset.

In [ ]:
#For example, this plot has a mean height of:
np.mean(points[:,2])

## Dataset Class

The first thing we need to develop to get a pytorch DL model up and running is a dataset class. This is used to load data into our model during training. We can define a simple one below.

In [160]:
class PointCloudDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, glob="*.npy"):
        #Read in all files and attach to the class
        self.files = list(Path(in_dir).glob(glob))        

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        points = np.load(self.files[idx])
        
        #Subset to xyz cols only
        points = points[:, :3]
        
        # Centralize coordinates
        points[:, 0:3] = points[:, 0:3] - np.mean(points[:, 0:3], axis=0)
        
        #Get the mean height
        target = np.mean(points[:,2])
        
        #Make correct shape to be tensor
        target = np.reshape(target, (1,1))[0]
        
        #Input data needs to be in tensor format
        torch.from_numpy(points).float().permute(1, 0)
        target = torch.tensor(target, dtype=torch.float64).float()
        
        print(target.shape)
        
        sample = {'points': points, 'target': target}

        return sample
        
#Lets test the Dataset class
dataset = PointCloudDataset(data_dir="data", glob="*.npy")

#Check length
print("Number of samples:", len(dataset))

#Check one sample
data_iter = iter(dataset)
sample = next(data_iter)

print("\nSample info:\n")
print("Type: ", type(sample))
print("Shape: ", len(sample))
print("Points shape: ", sample["points"].shape)
print("Target: ", sample['target'])

Number of samples: 12
torch.Size([1])

Sample info:

Type:  <class 'dict'>
Shape:  2
Points shape:  (7168, 3)
Target:  tensor([2.8866e-15])


We also need a class to load the data into the model in batches. A batch is a subset of data that passes through the model before the loss is calculated and the optimizer is implemented. For simplicity, we will use a batch size of 2.

In [161]:
#Set up data loader (loads data into the model during each iteration)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

#Test the loader
loader_iter = iter(loader)
batch = next(loader_iter)
print("Batch shape explained:")
print("Batch size, Num points, XYZ coords")
print(batch['points'].shape)
print(f"Target:\n{batch['target']}")

torch.Size([1])
torch.Size([1])
Batch shape explained:
Batch size, Num points, XYZ coords
torch.Size([2, 7168, 3])
Target:
tensor([[9.4528e-15],
        [3.8065e-16]])


## Define model

I asked ChatGPT to write me some code for a simple point cloud neural network and here it was it provided:

In [162]:
#Pytorch Implementation: https://github.com/WangYueFt/dgcnn
#Citation: Wang, Yue, et al. "Dynamic graph cnn for learning on point clouds." Acm Transactions On Graphics (tog) 38.5 (2019): 1-12.
def knn(x, k):
    inner = -2 * torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x ** 2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)

    idx = pairwise_distance.topk(k=k, dim=-1)[1]  # (batch_size, num_points, k)
    return idx


def get_graph_feature(x, k=20, idx=None):
    batch_size = x.size(0)
    num_points = x.size(2)
    x = x.view(batch_size, -1, num_points)
    if idx is None:
        idx = knn(x, k=k)  # (batch_size, num_points, k)
    device = x.device #Changed this from original torch.device('cuda') to try and get it to work on DDP

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1) * num_points

    idx = idx + idx_base

    idx = idx.view(-1)

    _, num_dims, _ = x.size()

    x = x.transpose(2,
                    1).contiguous()  # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size * num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims)
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)

    feature = torch.cat((feature - x, x), dim=3).permute(0, 3, 1, 2).contiguous()

    return feature


class DGCNN(nn.Module):
    def __init__(self, dropout=0.5, k=20, emb_dims=1024, n_outs=1):
        super(DGCNN, self).__init__()
        
        #Network config arguments
        self.k = k
        self.dropout = dropout
        self.emb_dims = emb_dims

        #Start of network
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(self.emb_dims)

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64 * 2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64 * 2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128 * 2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, self.emb_dims, kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.linear1 = nn.Linear(self.emb_dims * 2, 512, bias=False)
        self.bn6 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(p=self.dropout)
        self.linear2 = nn.Linear(512, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(p=self.dropout)
        self.linear3 = nn.Linear(256, n_outs)

    def forward(self, points):

        #For now, only use coordinates for DGCNN
        xyz = points[:, :2, :]

        batch_size = xyz.size(0)
        x = get_graph_feature(xyz, k=self.k)
        x = self.conv1(x)
        x1 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x1, k=self.k)
        x = self.conv2(x)
        x2 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x2, k=self.k)
        x = self.conv3(x)
        x3 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x3, k=self.k)
        x = self.conv4(x)
        x4 = x.max(dim=-1, keepdim=False)[0]

        x = torch.cat((x1, x2, x3, x4), dim=1)

        x = self.conv5(x)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)
        x = torch.cat((x1, x2), 1)

        x = F.leaky_relu(self.bn6(self.linear1(x)), negative_slope=0.2)
        x = self.dp1(x)
        x = F.leaky_relu(self.bn7(self.linear2(x)), negative_slope=0.2)
        x = self.dp2(x)
        x = self.linear3(x)

        return x

model = DGCNN()
print(model)

DGCNN(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (conv2): Sequential(
    (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=Fal

This is a VERY simple neural network that only has 2 layers, each with 64 neurons, for a total of 128 neurons.

Let's test the model:

In [165]:
model(batch['points'])

RuntimeError: selected index k out of range

The loss function is how the way you measure model accuracy. In this case since it is a regression problem, let use Mean Squared Error (MSE). 

The optimizer is a function that uses complex calculus and linear algebra to adjust the model neurons after each training iteration based on the loss. For example, if the loss is very large, the optimizer might make big changes to the network parameters, if it is small, then the opposite may occur.

The learning rate (lr) determines how fast/slow our optimizer helps our model learn. Larger learning rate = faster, but more chaotic learning.

In [ ]:
#Set up a loss function and an optimizer.
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

## Training Loop

Deep learning models are trained by iteratively passing the dataset through the network and then adjusting the neuron parameters (i.e., weights) based on each iteration. 

Once the entire training dataset has moved through the model, this is considered to be 1 epoch. 

Deep learning models are often trained for 100s or 1000s of epochs.

The dataloader will load one or several samples during each iteration within an epoch. The number of samples loaded during an iteration is determined by the batch size. Here we will use a batch size of 1. Batch size can be larger though. For example, ChatGPT was trained using a batch size of 2 Million pieces of text.

Once the batch is loaded, it is processed through the model, the model makes a prediction, the loss (i.e., error) is calculated, and then the optimizer is used to adjust the model parameters.

In [ ]:
for epoch in range(100):
    for step, batch in enumerate(loader):
        optimizer.zero_grad()
        pred = model(batch['points'])
        loss = loss_fn(pred, batch['target'])
        print(pred)
        print(batch['target'])
        loss.backward()
        optimizer.step()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 100, loss.item()))